In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Workspace, Model 

# make sure it is you own environment
ws = Workspace.get(name="MLOps_day7",
               subscription_id='5a60d53a-0d60-4174-a594-5c65a8fc1cbe',
               resource_group='MLOps')

env = Environment("ws_MLOps_env")

packages = CondaDependencies.create(conda_packages =['pip'],
                                    pip_packages = ['azureml-defaults', 'transformers', 'torch','torchvision','sklearn','wandb','datasets', 'joblib'])

env.python.conda_dependencies = packages

script_config =ScriptRunConfig(source_directory = '.',
                                script ='src/models/train_model.py',
                                arguments = ['--epochs',3, '--logger', 'azure'],
                                environment = env)

experiment = Experiment(workspace=ws, name = 'Training-170621')
run = experiment.submit(config = script_config)
run.wait_for_completion(show_output=True)

model = run.register_model(model_path='outputs/Roberta_fakenews_azure.pkl',model_name='Roberta_fakenews_azure')
# Create a model folder in the current directory
os.makedirs('./models', exist_ok=True)
# Download the model from run history
run.download_file(name='outputs/Roberta_fakenews_azure.pkl', output_file_path='./models/Roberta_fakenews_azure.pkl')


#model = Model.register(workspace = ws, model_name='model', model_path='outputs/model.pkl')
#for m in Model.list(ws):
    #print(m.name)

#model = run.register_model(model_path='outputs/model.pkl',model_name='model')
#model = ws.models['model']
#model = Model.register(workspace = ws, model_name='Roberta-fakenews', model_path='outputs/model.pkl')
# Create a model folder in the current directory
#os.makedirs('./models', exist_ok=True)
# Download the model from run history
#run.download_file(name='outputs/model.pt', output_file_path='./models/model.pt')
